In [2]:
import pandas as pd
from datetime import datetime, timedelta
import time
import numpy as np
import yfinance as yf

In [3]:
empresas = pd.read_csv('empresas.csv')
infos = pd.read_csv('infos_empresas.csv')
cotacoes = pd.read_csv('cotacoes.csv')
dividendos = pd.read_csv('dividendos.csv')

empresas = empresas.drop(empresas.columns[0], axis=1)
infos = infos.drop(infos.columns[0], axis=1)
cotacoes = cotacoes.drop(cotacoes.columns[0], axis=1)
dividendos = dividendos.drop(dividendos.columns[0], axis=1)

In [4]:
infos['value'] = infos.groupby(['cvm_code', 'data'])['value'].fillna(method='bfill')

C:\Users\eduar\AppData\Local\Temp\ipykernel_7492\3737537357.py:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  infos['value'] = infos.groupby(['cvm_code', 'data'])['value'].fillna(method='bfill')


In [5]:
infos['value'] = infos.groupby(['cvm_code', 'data'])['value'].fillna(method='ffill')

C:\Users\eduar\AppData\Local\Temp\ipykernel_7492\1645137256.py:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  infos['value'] = infos.groupby(['cvm_code', 'data'])['value'].fillna(method='ffill')


In [6]:
anos = np.arange(2015,2023)
anos

array([2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022])

In [7]:
empresas

,empresa,setor,codigo_cvm,estatal,acoes_on,acoes_pn,acoes_total,equivalencia_unit
0,CMIG4,energia,2453,True,735847624,1465523064,2201370688,1
1,CMIG3,energia,2453,True,735847624,1465523064,2201370688,1
2,TAEE11,energia,20257,False,590714069,442782652,1033496721,3
3,TAEE3,energia,20257,False,590714069,442782652,1033496721,1
4,TAEE4,energia,20257,False,590714069,442782652,1033496721,1
5,CPLE6,energia,14311,True,1300347300,1682463291,2982810591,1
6,CPLE3,energia,14311,True,1300347300,1682463291,2982810591,1
7,CSMG3,saneamento,19445,True,380253069,0,380253069,1
8,SAPR11,saneamento,18627,True,503735259,1007470260,1511205519,5
9,SAPR3,saneamento,18627,True,503735259,1007470260,1511205519,1


In [8]:
df_list = []
for i in range(len(empresas)):
    ticker = empresas.iloc[i]['empresa']
    cod_cvm = empresas.iloc[i]['codigo_cvm']
    n_de_acoes = empresas.iloc[i]['acoes_total']
    setor = empresas.iloc[i]['setor']
    estatal = empresas.iloc[i]['estatal']
    equivalencia_unit = empresas.iloc[i]['equivalencia_unit']
    
    for ano in anos:
        dados_empresa = infos[infos['cvm_code']==cod_cvm]
        dados_empresa_ano = dados_empresa[dados_empresa['year']==ano]

        try:
            lucro_liquido = dados_empresa_ano[dados_empresa_ano['data']=='net_income']['value'].values[0]
        except:
            lucro_liquido = 1
        try:
            patrimonio_liquido = dados_empresa_ano[dados_empresa_ano['data']=='equity']['value'].values[0]
        except:
            patrimonio_liquido = 1
        try:
            divida_liquida = dados_empresa_ano[dados_empresa_ano['data']=='net_debt']['value'].values[0]
        except:
            divida_liquida = 1

        roe = lucro_liquido/patrimonio_liquido
        debt_over_income = divida_liquida/lucro_liquido
        vpa = (patrimonio_liquido/n_de_acoes) * equivalencia_unit
        lpa = (lucro_liquido/n_de_acoes) * equivalencia_unit

        dicionario = {
            'ticker' : ticker,
            'cvm_code' : cod_cvm,
            'setor' : setor,
            'estatal' : estatal,
            'year' : ano,
            'roe' : roe,
            'debt_ratio' : debt_over_income,
            'vpa' : vpa,
            'lpa' : lpa
        }

        df = pd.DataFrame(dicionario, index=[0])
        df_list.append(df)

C:\Users\eduar\AppData\Local\Temp\ipykernel_7492\2011381830.py:27: RuntimeWarning: divide by zero encountered in scalar divide
  roe = lucro_liquido/patrimonio_liquido
C:\Users\eduar\AppData\Local\Temp\ipykernel_7492\2011381830.py:27: RuntimeWarning: invalid value encountered in scalar divide
  roe = lucro_liquido/patrimonio_liquido
C:\Users\eduar\AppData\Local\Temp\ipykernel_7492\2011381830.py:28: RuntimeWarning: divide by zero encountered in scalar divide
  debt_over_income = divida_liquida/lucro_liquido


In [9]:
indicadores = pd.concat(df_list)
indicadores

,ticker,cvm_code,setor,estatal,year,roe,debt_ratio,vpa,lpa
0,CMIG4,2453,energia,True,2015,0.190104,4.785397,5.899813,1.121575
0,CMIG4,2453,energia,True,2016,0.025881,39.342204,5.875599,0.152066
0,CMIG4,2453,energia,True,2017,0.069894,12.289442,6.509642,0.454987
0,CMIG4,2453,energia,True,2018,0.109271,7.565812,7.240637,0.791195
0,CMIG4,2453,energia,True,2019,0.198376,4.226188,7.314763,1.451075
...,...,...,...,...,...,...,...,...,...
0,TIMS3,24929,telecom,False,2018,0.079137,-1.500065,8.005929,0.633563
0,TIMS3,24929,telecom,False,2019,NaN,-inf,0.000000,0.000000
0,TIMS3,24929,telecom,False,2020,0.079529,-1.247876,9.576464,0.761602
0,TIMS3,24929,telecom,False,2021,NaN,inf,0.000000,0.000000


In [18]:
indicadores.head(30)

,ticker,cvm_code,setor,estatal,year,roe,debt_ratio,vpa,lpa,dividendo
0,CMIG4,2453,energia,True,2015,0.190104,4.785397,5.899813,1.121575,1.219627
0,CMIG4,2453,energia,True,2016,0.025881,39.342204,5.875599,0.152066,1.293778
0,CMIG4,2453,energia,True,2017,0.069894,12.289442,6.509642,0.454987,0.487119
0,CMIG4,2453,energia,True,2018,0.109271,7.565812,7.240637,0.791195,1.288606
0,CMIG4,2453,energia,True,2019,0.198376,4.226188,7.314763,1.451075,1.450221
0,CMIG4,2453,energia,True,2020,0.163933,3.483235,7.939303,1.301517,1.228649
0,CMIG4,2453,energia,True,2021,0.192833,2.348781,8.840749,1.704787,2.352093
0,CMIG4,2453,energia,True,2022,0.187959,1.805967,9.895319,1.859917,3.064526
0,CMIG3,2453,energia,True,2015,0.190104,4.785397,5.899813,1.121575,1.219627
0,CMIG3,2453,energia,True,2016,0.025881,39.342204,5.875599,0.152066,1.293778


In [10]:
dividendos_list = []
for i in range(len(indicadores)):
    ticker = indicadores.iloc[i]['ticker']
    ano = indicadores.iloc[i]['year']

    dividendos_ticker_ano = dividendos[dividendos['ticker']==ticker][dividendos[dividendos['ticker']==ticker]['year']==ano]
    dividendo_ano_valor = dividendos_ticker_ano['value'].sum()

    dividendos_list.append(dividendo_ano_valor)

In [11]:
indicadores['dividendo'] = dividendos_list

In [12]:
indicadores

,ticker,cvm_code,setor,estatal,year,roe,debt_ratio,vpa,lpa,dividendo
0,CMIG4,2453,energia,True,2015,0.190104,4.785397,5.899813,1.121575,1.219627
0,CMIG4,2453,energia,True,2016,0.025881,39.342204,5.875599,0.152066,1.293778
0,CMIG4,2453,energia,True,2017,0.069894,12.289442,6.509642,0.454987,0.487119
0,CMIG4,2453,energia,True,2018,0.109271,7.565812,7.240637,0.791195,1.288606
0,CMIG4,2453,energia,True,2019,0.198376,4.226188,7.314763,1.451075,1.450221
...,...,...,...,...,...,...,...,...,...,...
0,TIMS3,24929,telecom,False,2018,0.079137,-1.500065,8.005929,0.633563,0.236894
0,TIMS3,24929,telecom,False,2019,NaN,-inf,0.000000,0.000000,0.465888
0,TIMS3,24929,telecom,False,2020,0.079529,-1.247876,9.576464,0.761602,0.549765
0,TIMS3,24929,telecom,False,2021,NaN,inf,0.000000,0.000000,0.432831


In [13]:
equivalencia_empresas_ticker_list = []
equivalencia_empresas_valor_list = []
for cvm_code in empresas['codigo_cvm'].unique():
    ticker_list = empresas[empresas['codigo_cvm']==cvm_code]['empresa'].values
    ticker_principal = ticker_list[0]
    
    if len(ticker_list)==1:
        equivalencia_empresas_ticker_list.append(ticker_principal)
        equivalencia_empresas_valor_list.append(1)
    else:
        for ticker in ticker_list:
            dividendos_ticker = indicadores[indicadores['ticker']==ticker]['dividendo'].mean()
            dividendos_ticker_principal = indicadores[indicadores['ticker']==ticker_principal]['dividendo'].mean()
            multiplicador = dividendos_ticker/dividendos_ticker_principal

            equivalencia_empresas_ticker_list.append(ticker_principal)
            equivalencia_empresas_valor_list.append(multiplicador)


In [14]:
empresas['equivalencia_ticker'] = equivalencia_empresas_ticker_list
empresas['equivalencia_valor'] = equivalencia_empresas_valor_list

In [15]:
empresas

,empresa,setor,codigo_cvm,estatal,acoes_on,acoes_pn,acoes_total,equivalencia_unit,equivalencia_ticker,equivalencia_valor
0,CMIG4,energia,2453,True,735847624,1465523064,2201370688,1,CMIG4,1.000000
1,CMIG3,energia,2453,True,735847624,1465523064,2201370688,1,CMIG4,0.924145
2,TAEE11,energia,20257,False,590714069,442782652,1033496721,3,TAEE11,1.000000
3,TAEE3,energia,20257,False,590714069,442782652,1033496721,1,TAEE11,0.337737
4,TAEE4,energia,20257,False,590714069,442782652,1033496721,1,TAEE11,0.337737
5,CPLE6,energia,14311,True,1300347300,1682463291,2982810591,1,CPLE6,1.000000
6,CPLE3,energia,14311,True,1300347300,1682463291,2982810591,1,CPLE6,0.908667
7,CSMG3,saneamento,19445,True,380253069,0,380253069,1,CSMG3,1.000000
8,SAPR11,saneamento,18627,True,503735259,1007470260,1511205519,5,SAPR11,1.000000
9,SAPR3,saneamento,18627,True,503735259,1007470260,1511205519,1,SAPR11,0.290275


In [16]:
empresas.to_csv('empresas_v2.csv')
indicadores.to_csv('indicadores.csv')